# Audioset

# Setup

In [1]:
from tqdm import tqdm
import tensorflow as tf
from utils import vggish_slim
from utils import vggish_input
from utils import vggish_params
from utils import vggish_postprocess

In [2]:
import numpy as np
import pandas as pd

items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

In [3]:
items

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,...,Thriller,War,Western,Summary,Cast,Director,Rating,Runtime,No. of ratings,YT-Trailer ID
0,0,0,0,1,Toy Story (1995),01-Jan-95,NaN,https://www.imdb.com/title/tt0114709/,0,0,...,0,0,0,A little boy named Andy loves to be in his roo...,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,8.3,81.0,930K,mGXHH9iAfLA
1,1,1,1,2,GoldenEye (1995),01-Jan-95,NaN,https://www.imdb.com/title/tt0113189/,0,1,...,1,0,0,When a deadly satellite weapon system falls in...,Pierce Brosnan|Sean Bean|Izabella Scorupco|Fam...,Martin Campbell,7.2,130.0,248K,IWFtzQOX_nY
2,2,2,2,3,Four Rooms (1995),01-Jan-95,NaN,https://www.imdb.com/title/tt0113101/,0,0,...,1,0,0,This movie features the collaborative director...,Sammi Davis|Amanda De Cadenet|Valeria Golino|M...,Allison Anders,6.8,98.0,102K,hugpRyTc1fE
3,3,3,3,4,Get Shorty (1995),01-Jan-95,NaN,https://www.imdb.com/title/tt0113161/,0,1,...,0,0,0,"Some guys get all the luck, whether they like ...",John Travolta|Gene Hackman|Rene Russo|Danny De...,Barry Sonnenfeld,6.9,105.0,81K,yvsnu3crV8g
4,4,4,4,5,Copycat (1995),01-Jan-95,NaN,https://www.imdb.com/title/tt0112722/,0,0,...,1,0,0,"In San Francisco, the criminal psychologist He...",Sigourney Weaver|Holly Hunter|Dermot Mulroney|...,Jon Amiel,6.6,123.0,57K,gMrMLbxEt5Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1677,1677,1677,1678,Mat' i syn (1997),06-Feb-98,NaN,https://www.imdb.com/title/tt0119711/,0,0,...,0,0,0,A slow and poignant story of love and patience...,Aleksei Ananishnov|Gudrun Geyer,Aleksandr Sokurov,7.4,73.0,3.9K,NaN
1678,1678,1678,1678,1679,B. Monkey (1998),06-Feb-98,NaN,https://www.imdb.com/title/tt0120594/,0,0,...,1,0,0,Alan Furnace is a young man with the perfectly...,Asia Argento|Jared Harris|Rupert Everett|Jonat...,Michael Radford,5.9,92.0,3.4K,NaN
1679,1679,1679,1679,1680,Sliding Doors (1998),01-Jan-98,NaN,https://www.imdb.com/title/tt0120148/,0,0,...,0,0,0,"In London, the public relation Helen is fired ...",Gwyneth Paltrow|John Hannah|John Lynch|Jeanne ...,Peter Howitt,6.7,99.0,66K,NaN
1680,1680,1680,1680,1681,You So Crazy (1994),01-Jan-94,NaN,https://www.imdb.com/title/tt0111804/,0,0,...,0,0,0,"Stand up comedy by Martin Lawrence, filmed in ...",Martin Lawrence,Thomas Schlamme,6.6,84.0,952,NaN


# Extract Audio Features

In [ ]:
embeddings = {}
with tf.Graph().as_default(), tf.Session() as sess:
    # Define the model in inference mode, load the checkpoint, and
    # locate input and output tensors.
    vggish_slim.define_vggish_slim(training=False)
    vggish_slim.load_vggish_slim_checkpoint(sess, "./pretrained/vggish_model.ckpt")
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.OUTPUT_TENSOR_NAME)
    
    for index, movie in items.iterrows():
        try:
            examples_batch = vggish_input.wavfile_to_examples("../Datasets/ml-100k/Audio/{}.wav".format(movie['YT-Trailer ID']))
            [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
            if(movie['YT-Trailer ID'] == movie['YT-Trailer ID']):
                embeddings[movie['YT-Trailer ID']] = np.mean(embedding_batch, axis = 0)
                print(len(embeddings))
        except:
            pd.DataFrame(embeddings).to_csv("../Datasets/ml-100k/Audio/embeddings.csv")
            continue


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.


2021-11-12 13:42:09.453721: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-12 13:42:09.483690: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2021-11-12 13:42:09.484024: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557e35590710 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-12 13:42:09.484047: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 8 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info 

INFO:tensorflow:Restoring parameters from ./pretrained/vggish_model.ckpt


OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26653 thread 1 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26668 thread 2 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26669 thread 3 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26670 thread 4 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26672 thread 6 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26671 thread 5 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26673 thread 7 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26674 thread 8 bound to OS proc set 0


1
2
3


OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26654 thread 9 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26706 thread 11 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26705 thread 10 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26707 thread 12 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26708 thread 13 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26709 thread 14 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26711 thread 16 bound to OS proc set 0
OMP: Info #254: KMP_AFFINITY: pid 26616 tid 26710 thread 15 bound to OS proc set 7


4
5
6
7
8
9
10
11


In [ ]:
pd.DataFrame(embeddings).to_csv("../Datasets/ml-100k/Audio/embeddings.csv")